In [1]:
#Scraping xe.gr
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import date
from zenrows import ZenRowsClient
from IPython.display import display, HTML


In [2]:

# RENTALS 
# Set the user-agent header
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

# Create a proxy client
client = ZenRowsClient("a45285ef2974d1cf4c9f103296a7835b19844643")
BASE_URL='https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic'
response = client.get(BASE_URL, headers=headers)
soup=BeautifulSoup(response.content,"html.parser")

# PAGINATION : Finding all li tags in ul and printing the text within it 
web_urls=[]
nums=[]

buttons=soup.find('ul',class_='results-pagination')
for li in buttons.find_all('li'):
    nums.append(li.text.replace('\n',''))

# Take the last element from the list of texts found in li elements
last_page=int(nums[-1])
print(last_page*34)


#Create the urls of multiple "xe" result pages, 
for i in range(1,last_page+1):
    web_url_base="https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page="
    web_urls.append(web_url_base+str(i))
    
print(web_urls)



4590
['https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=1', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=2', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=3', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=4', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=5', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=6', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=7', 'https:

In [7]:
#SALES

# Create a proxy client
BASE_URL_Sales='https://www.xe.gr/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic'
response_Sales = client.get(BASE_URL_Sales, headers=headers)
soup_Sales=BeautifulSoup(response_Sales.content,"html.parser")

# PAGINATION : Finding all li tags in ul and printing the text within it 
num_Sales=[]
web_urls_Sales=[]
buttons_Sales=soup_Sales.find('ul',class_='results-pagination')
for li in buttons_Sales.find_all('li'):
    num_Sales.append(li.text.replace('\n',''))

# Take the last element from the list of texts found in li elements
last_page_Sales=int(num_Sales[-1])
print(last_page_Sales)

num_of_ads_sales=last_page_Sales*34
print(num_of_ads_sales)


for i in range(1,last_page_Sales+1):
    web_url_Sales_base="https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page="
    web_urls_Sales.append(web_url_Sales_base+str(i))
    
    
print(web_urls_Sales)



295
10030
['https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=1', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=2', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=3', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=4', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=5', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=6', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=7', 'https://

In [4]:
# RENTALS PAGES
# Access page source code, parse, find elements and write them in csv file 
price_list=[]
title_list=[]
level_list=[]
url_list=[]
address_list=[]
price_sqm_list=[]
construction_year_list=[]  

for web_url in web_urls:
    try:
        response=client.get(web_url,headers=headers)
    
        xe_webpage=response.text
    
        soup=BeautifulSoup(xe_webpage,"html.parser")
        print('OK')
    except:
        print(f"No response for {web_url}")

    
    
    prices=soup.find_all(name="span",class_="property-ad-price")
    titles=soup.find_all(name="div",class_="common-property-ad-title")
    
    url_element=soup.find_all(name="div",class_="common-property-ad-body grid-y align-justify")
    addresses=soup.find_all(name="span",class_="common-property-ad-address")
    prices_sqm=soup.find_all(name="span",class_="property-ad-price-per-sqm")
    details=soup.find_all(name='div',class_="common-property-ad-details grid-x")



    
    for price in prices:
        p=re.findall(r'\d+', price.getText())
        price_list.append(int("".join(p)))
       #price_list.append(price.getText().replace("\xa0"," ")) 
    
    for title in titles:
        t=re.findall(r'\d+', title.getText())
        title_list.append(int("".join(t)))
        #title_list.append(title.getText().replace("\n",""))
    
        
    for urlz in url_element:
        urls=urlz.find('a')
        url_list.append(urls.get('href'))
    
    for address in addresses:
        address_clean=re.findall(r'\((.*?)\)',address.getText())
        try:
            address_list.append(address_clean[0])
        except:
            address_list.append('None')
    
    for price_sqm in prices_sqm:
        psqm=re.findall(r'\d+', price_sqm.getText())
        price_sqm_list.append(int("".join(psqm)))

    for text in details:
        levels=text.find_all(name="span",class_="property-ad-level")
        construction_years=text.find_all(name='div',class_="grid-x property-ad-construction-year-container")
        if construction_years:
            for years in construction_years:
                year=re.findall(r'\d+',years.getText())
                construction_year_list.append(int("".join(year)))
        else:
            construction_year_list.append("None")

        # if levels:
        #     for level in levels:
        #         l=re.findall(r'\d\w+',level.getText())
        #     else:
        #         level_list.append('None')
                   
#print(price_list)
#print(title_list)
#print(level_list)
#print(url_list)
#print(address_list)
#print(price_sqm_list)
print(len(price_list))
print(len(title_list))
print(len(level_list))
print(len(url_list))
print(len(address_list))
print(len(price_sqm_list))
print(len(construction_year_list))

    
    
    
#Create dictionary with column names as keys
dict={'Sqm':title_list,'Price':price_list,'URL':url_list, 'neighbourhood':address_list, 'price_sq/m':price_sqm_list, 'construction_year':construction_year_list, "date_imported":date.today()}
df=pd.DataFrame(dict)
#create dataframe
#df=pd.DataFrame(list(zip(price_list,title_list,url_list,address_list,price_sqm_list)), columns=['price','title','url','address','price_sqm'

#duplicateRows = df[df.duplicated(['Title', 'Price','neighbourhood'])]
#write to csv file
df.to_csv(f'xe_rentals_{date.today()}',mode='a',index=False, header=False)

time.sleep(1)


OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
4562
4562
0
4562
4562
4562
4562


In [5]:
display(df)

,Sqm,Price,URL,neighbourhood,price_sq/m,construction_year,date_imported
0,55,600,https://www.xe.gr/en/property/d/property-to-re...,Ano Kipseli,11,1955,2024-01-25
1,36,480,https://www.xe.gr/en/property/d/property-to-re...,Ippokratous,13,None,2024-01-25
2,85,1000,https://www.xe.gr/en/property/d/property-to-re...,Koukaki,12,2000,2024-01-25
3,71,820,https://www.xe.gr/en/property/d/property-to-re...,Erythros,12,None,2024-01-25
4,53,470,https://www.xe.gr/en/property/d/property-to-re...,Sepolia,9,1983,2024-01-25
...,...,...,...,...,...,...,...
4557,75,650,https://www.xe.gr/en/property/d/property-to-re...,None,9,1979,2024-01-25
4558,340,7000,https://www.xe.gr/en/property/d/property-to-re...,None,21,None,2024-01-25
4559,70,680,https://www.xe.gr/en/property/d/property-to-re...,Center,10,1975,2024-01-25
4560,50,590,https://www.xe.gr/en/property/d/property-to-re...,Agora,12,1954,2024-01-25


In [8]:
price_list=[]
title_list=[]
level_list=[]
url_list=[]
address_list=[]
price_sqm_list=[]
constru_year_list=[]


for web_url in web_urls_Sales[1:4]:
    try:
        response=client.get(web_url,headers=headers)
    
        xe_webpage=response.text
    
        soup=BeautifulSoup(xe_webpage,"html.parser")
        print('OK')
    except:
        print(f"No response for {web_url}")

    
    
    prices=soup.find_all(name="span",class_="property-ad-price")
    titles=soup.find_all(name="div",class_="common-property-ad-title")
    levels=soup.find_all(name="span",class_="property-ad-level")
    url_element=soup.find_all(name="div",class_="common-property-ad-body grid-y align-justify")
    addresses=soup.find_all(name="span",class_="common-property-ad-address")
    prices_sqm=soup.find_all(name="span",class_="property-ad-price-per-sqm")
    #construction_dates=soup.find_all(name="div",class_="grid-x property-ad-construction-year-container")

    
    for price in prices:
        p=re.findall(r'\d+', price.getText())
        price_list.append(int("".join(p)))
       #price_list.append(price.getText().replace("\xa0"," ")) 
    
    for title in titles:
        t=re.findall(r'\d+', title.getText())
        title_list.append(int("".join(t)))
        #title_list.append(title.getText().replace("\n",""))
    
    for level in levels:
        y1=level.getText().replace("\n","")
        level_list.append(y1.replace(" ", ""))
        
    for urlz in url_element:
        urls=urlz.find('a')
        url_list.append(urls.get('href'))
    
    for address in addresses:
        address_clean=re.findall(r'\((.*?)\)',address.getText())
        try:
            address_list.append(address_clean[0])
        except:
            address_list.append('NA')
        #address_clean=address.getText().replace(" ", "")
        #address_list.append(address_clean.replace("\n",""))  
    
    for price_sqm in prices_sqm:
        psqm=re.findall(r'\d+', price_sqm.getText())
        price_sqm_list.append(int("".join(psqm)))
        #price_sqm_list.append((price_sqm.getText().replace("\xa0","")).replace("\n",""))

    # for constru_date in construction_dates:
    #     try:
    #         year=re.findall(r'\d+',constru_date.getText())
    #         constru_year_list.append(int("".join(year)))
    #     except:
    #         constru_year_list.append("None")
    
print(len(price_list))
print(len(title_list))
print(len(level_list))
print(len(url_list))
print(len(address_list))
print(len(price_sqm_list))

    
    
    
#Create dictionary with column names as keys
dict={'Sqm':title_list,'Price':price_list,'URL':url_list, 'neighbourhood':address_list, 'price_sq/m':price_sqm_list}

#create dataframe
df=pd.DataFrame(dict)
#df=pd.DataFrame(list(zip(price_list,title_list,url_list,address_list,price_sqm_list)), columns=['price','title','url','address','price_sqm'
#duplicateRows = df[df.duplicated(['Title', 'Price','neighbourhood'])]


#write to csv file
df.to_csv(f'xe_Sales_{date.today()}',mode='a',index=False, header=False)

time.sleep(1)

OK
OK
OK
102
102
109
102
102
102


In [47]:
display(df)

,Sqm,Price,URL,level,neighbourhood,price_sq/m,construction_year,date_imported
0,180,1200,https://www.xe.gr/en/property/d/property-to-re...,1st,Agia Eleousa,7,1970,2024-01-24
1,26,500,https://www.xe.gr/en/property/d/property-to-re...,,Agora,19,1980,2024-01-24
2,88,650,https://www.xe.gr/en/property/d/property-to-re...,4th,Mykoniatika,7,2008,2024-01-24
3,68,580,https://www.xe.gr/en/property/d/property-to-re...,2nd,Center,9,1972,2024-01-24
4,45,550,https://www.xe.gr/en/property/d/property-to-re...,2nd,Center,12,1969,2024-01-24
...,...,...,...,...,...,...,...,...
335,55,500,https://www.xe.gr/en/property/d/property-to-re...,2nd,Center,9,1980,2024-01-24
336,81,680,https://www.xe.gr/en/property/d/property-to-re...,5th,Center,8,1972,2024-01-24
337,80,600,https://www.xe.gr/en/property/d/property-to-re...,,Skopeftirio,8,1970,2024-01-24
338,34,480,https://www.xe.gr/en/property/d/property-to-re...,,OTE,14,1967,2024-01-24


In [64]:
print(len(construction_dates))
print(len(title_list))

27
68
